## Logistic Regression

In [45]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
from scipy.sparse import csr_matrix, vstack, hstack
from sklearn.model_selection import GridSearchCV

### Data Prep for ML

In [46]:
train = pd.read_pickle("train.pkl").dropna()
test = pd.read_pickle("test.pkl").dropna()

In [47]:
#One hot encoding
encoded_train = pd.get_dummies(train, columns = ['weighted_parent_sentiment_score','weighted_comment_sentiment_score'], drop_first=True)
encoded_test = pd.get_dummies(test, columns = ['weighted_parent_sentiment_score','weighted_comment_sentiment_score'], drop_first=True)

In [48]:
train_parent_tdidf_csr = vstack(encoded_train["parent_comment_tdidf"])
test_parent_tdidf_csr = vstack(encoded_test["parent_comment_tdidf"])

train_tdidf_csr = vstack(encoded_train["comment_tdidf"])
test_tdidf_csr = vstack(encoded_test["comment_tdidf"])

train_parent_bow_csr = vstack(encoded_train["parent_comment_bow"])
test_parent_bow_csr = vstack(encoded_test["parent_comment_bow"])

train_bow_csr = vstack(encoded_train["comment_bow"])
test_bow_csr = vstack(encoded_test["comment_bow"])

In [49]:
encoded_train.columns

Index(['comment', 'author', 'subreddit', 'score', 'ups', 'downs', 'date',
       'created_utc', 'parent_comment', 'comment_tokens',
       'parent_comment_tokens', 'comment_score', 'parent_comment_score',
       'comment_word_count', 'parent_comment_word_count',
       'comment_token_count', 'parent_comment_token_count',
       'comment_unique_word_count', 'parent_comment_unique_word_count',
       'comment_unique_token_count', 'parent_comment_unique_token_count',
       'comment_stopword_count', 'parent_comment_stopword_count',
       'comment_mean_word_length', 'parent_comment_mean_word_length',
       'comment_mean_token_length', 'parent_comment_mean_token_length',
       'comment_char_count', 'parent_comment_char_count',
       'comment_punctuation_count', 'parent_comment_punctuation_count',
       'comment_hashtag_count', 'parent_comment_hashtag_count',
       'comment_number_count', 'parent_comment_number_count', 'comment_bow',
       'parent_comment_bow', 'comment_tdidf', 'paren

In [50]:
list_of_features = ['score','ups','downs',
       'comment_word_count','parent_comment_word_count',
       'comment_token_count', 'parent_comment_token_count',
       'comment_unique_word_count', 'parent_comment_unique_word_count',
       'comment_unique_token_count', 'parent_comment_unique_token_count',
       'comment_stopword_count', 'parent_comment_stopword_count',
       'comment_mean_word_length', 'parent_comment_mean_word_length',
       'comment_mean_token_length', 'parent_comment_mean_token_length',
       'comment_char_count', 'parent_comment_char_count',
       'comment_punctuation_count', 'parent_comment_punctuation_count',
       'comment_hashtag_count', 'parent_comment_hashtag_count',
       'comment_number_count', 'parent_comment_number_count',
       'weighted_parent_sentiment_score_neutral',
       'weighted_parent_sentiment_score_positive',
       'weighted_comment_sentiment_score_neutral',
       'weighted_comment_sentiment_score_positive']

In [51]:
X_train_gen_features = csr_matrix(encoded_train[list_of_features])
y_train_LR = encoded_train['label']

X_test_gen_features = csr_matrix(encoded_test[list_of_features])
y_test_LR = encoded_test['label']

### Baseline 1: General Features

In [68]:
#Data Specific Prep
X_train_LR = X_train_gen_features

scaler = MaxAbsScaler()
X_train_LR = scaler.fit_transform(X_train_LR)


In [69]:
k = 5

model = LogisticRegression(max_iter = 500)
kf = KFold(n_splits=k, shuffle=True, random_state=42)

cross_val_scores = []
for train_index, val_index in kf.split(X_train_LR): 
    X_train, X_val = X_train_LR[train_index], X_train_LR[val_index]
    y_train, y_val = y_train_LR.iloc[train_index,], y_train_LR.iloc[val_index,]
    
    # Train the model on the training data
    model.fit(X_train, y_train)
    
    # Make predictions on the validation data
    y_pred = model.predict(X_val)
    
    # Calculate accuracy and store it in the list
    accuracy = accuracy_score(y_val, y_pred)
    cross_val_scores.append(accuracy)

mean_accuracy = sum(cross_val_scores) / k
std_accuracy = np.std(cross_val_scores)

print(f"Mean Accuracy: {mean_accuracy}")
print(f"Standard Deviation of Accuracy: {std_accuracy}")

Mean Accuracy: 0.549073778456603
Standard Deviation of Accuracy: 0.0011866425387839022


### Baseline 2: Comment BoW

In [70]:
#Data Specific Prep
X_train_LR = train_bow_csr

scaler = MaxAbsScaler()
X_train_LR = scaler.fit_transform(X_train_LR)

In [71]:
k = 5

model = LogisticRegression(max_iter = 500)
kf = KFold(n_splits=k, shuffle=True, random_state=42)

cross_val_scores = []
for train_index, val_index in kf.split(X_train_LR): 
    X_train, X_val = X_train_LR[train_index], X_train_LR[val_index]
    y_train, y_val = y_train_LR.iloc[train_index,], y_train_LR.iloc[val_index,]
    
    # Train the model on the training data
    model.fit(X_train, y_train)
    
    # Make predictions on the validation data
    y_pred = model.predict(X_val)
    
    # Calculate accuracy and store it in the list
    accuracy = accuracy_score(y_val, y_pred)
    cross_val_scores.append(accuracy)

mean_accuracy = sum(cross_val_scores) / k
std_accuracy = np.std(cross_val_scores)

print(f"Mean Accuracy: {mean_accuracy}")
print(f"Standard Deviation of Accuracy: {std_accuracy}")

Mean Accuracy: 0.6559869405583454
Standard Deviation of Accuracy: 0.002837919243242117


### Baseline 3: Comment TD-IDF

In [72]:
#Data Specific Prep
X_train_LR = train_tdidf_csr

scaler = MaxAbsScaler()
X_train_LR = scaler.fit_transform(X_train_LR)

In [73]:
k = 5

model = LogisticRegression(max_iter = 500)
kf = KFold(n_splits=k, shuffle=True, random_state=42)

cross_val_scores = []
for train_index, val_index in kf.split(X_train_LR): 
    X_train, X_val = X_train_LR[train_index], X_train_LR[val_index]
    y_train, y_val = y_train_LR.iloc[train_index,], y_train_LR.iloc[val_index,]
    
    # Train the model on the training data
    model.fit(X_train, y_train)
    
    # Make predictions on the validation data
    y_pred = model.predict(X_val)
    
    # Calculate accuracy and store it in the list
    accuracy = accuracy_score(y_val, y_pred)
    cross_val_scores.append(accuracy)

mean_accuracy = sum(cross_val_scores) / k
std_accuracy = np.std(cross_val_scores)

print(f"Mean Accuracy: {mean_accuracy}")
print(f"Standard Deviation of Accuracy: {std_accuracy}")

Mean Accuracy: 0.6562352661068654
Standard Deviation of Accuracy: 0.0017712936494375434


### Baseline 4: General Features + BoW

In [74]:
X_train_LR = hstack([X_train_gen_features,train_bow_csr])

scaler = MaxAbsScaler()
X_train_LR = scaler.fit_transform(X_train_LR)

In [75]:
k = 5

model = LogisticRegression(max_iter = 500)
kf = KFold(n_splits=k, shuffle=True, random_state=42)

cross_val_scores = []
for train_index, val_index in kf.split(X_train_LR): 
    X_train, X_val = X_train_LR[train_index], X_train_LR[val_index]
    y_train, y_val = y_train_LR.iloc[train_index,], y_train_LR.iloc[val_index,]
    
    # Train the model on the training data
    model.fit(X_train, y_train)
    
    # Make predictions on the validation data
    y_pred = model.predict(X_val)
    
    # Calculate accuracy and store it in the list
    accuracy = accuracy_score(y_val, y_pred)
    cross_val_scores.append(accuracy)

mean_accuracy = sum(cross_val_scores) / k
std_accuracy = np.std(cross_val_scores)

print(f"Mean Accuracy: {mean_accuracy}")
print(f"Standard Deviation of Accuracy: {std_accuracy}")

Mean Accuracy: 0.6607484256002715
Standard Deviation of Accuracy: 0.0023205634651901974


### Baseline 5: General Features + TD-IDF

In [76]:
X_train_LR = hstack([X_train_gen_features,train_tdidf_csr])

scaler = MaxAbsScaler()
X_train_LR = scaler.fit_transform(X_train_LR)

In [77]:
k = 5

model = LogisticRegression(max_iter = 500)
kf = KFold(n_splits=k, shuffle=True, random_state=42)

cross_val_scores = []
for train_index, val_index in kf.split(X_train_LR): 
    X_train, X_val = X_train_LR[train_index], X_train_LR[val_index]
    y_train, y_val = y_train_LR.iloc[train_index,], y_train_LR.iloc[val_index,]
    
    # Train the model on the training data
    model.fit(X_train, y_train)
    
    # Make predictions on the validation data
    y_pred = model.predict(X_val)
    
    # Calculate accuracy and store it in the list
    accuracy = accuracy_score(y_val, y_pred)
    cross_val_scores.append(accuracy)

mean_accuracy = sum(cross_val_scores) / k
std_accuracy = np.std(cross_val_scores)

print(f"Mean Accuracy: {mean_accuracy}")
print(f"Standard Deviation of Accuracy: {std_accuracy}")

Mean Accuracy: 0.6611209176806286
Standard Deviation of Accuracy: 0.0020015926105550713


### Baseline 6: Gen Features + Comment TDIDF + Parent TDIDF

In [78]:
X_train_LR = hstack([X_train_gen_features,train_tdidf_csr,train_parent_tdidf_csr])

scaler = MaxAbsScaler()
X_train_LR = scaler.fit_transform(X_train_LR)

In [79]:
k = 5

model = LogisticRegression(max_iter = 500)
kf = KFold(n_splits=k, shuffle=True, random_state=42)

cross_val_scores = []
for train_index, val_index in kf.split(X_train_LR): 
    X_train, X_val = X_train_LR[train_index], X_train_LR[val_index]
    y_train, y_val = y_train_LR.iloc[train_index,], y_train_LR.iloc[val_index,]
    
    # Train the model on the training data
    model.fit(X_train, y_train)
    
    # Make predictions on the validation data
    y_pred = model.predict(X_val)
    
    # Calculate accuracy and store it in the list
    accuracy = accuracy_score(y_val, y_pred)
    cross_val_scores.append(accuracy)

mean_accuracy = sum(cross_val_scores) / k
std_accuracy = np.std(cross_val_scores)

print(f"Mean Accuracy: {mean_accuracy}")
print(f"Standard Deviation of Accuracy: {std_accuracy}")

Mean Accuracy: 0.6544101625725147
Standard Deviation of Accuracy: 0.0036873570540180743


### Baseline 7: Gen + Parent TDIDF

In [80]:
X_train_LR = hstack([X_train_gen_features,train_parent_tdidf_csr])

scaler = MaxAbsScaler()
X_train_LR = scaler.fit_transform(X_train_LR)

In [81]:
k = 5

model = LogisticRegression(max_iter = 500)
kf = KFold(n_splits=k, shuffle=True, random_state=42)

cross_val_scores = []
for train_index, val_index in kf.split(X_train_LR): 
    X_train, X_val = X_train_LR[train_index], X_train_LR[val_index]
    y_train, y_val = y_train_LR.iloc[train_index,], y_train_LR.iloc[val_index,]
    
    # Train the model on the training data
    model.fit(X_train, y_train)
    
    # Make predictions on the validation data
    y_pred = model.predict(X_val)
    
    # Calculate accuracy and store it in the list
    accuracy = accuracy_score(y_val, y_pred)
    cross_val_scores.append(accuracy)

mean_accuracy = sum(cross_val_scores) / k
std_accuracy = np.std(cross_val_scores)

print(f"Mean Accuracy: {mean_accuracy}")
print(f"Standard Deviation of Accuracy: {std_accuracy}")

Mean Accuracy: 0.568635008845028
Standard Deviation of Accuracy: 0.0013539634505594297


### Hyper Parameter Tuning with Grid Search CV

In [82]:
X_train_LR = hstack([X_train_gen_features,train_tdidf_csr])

scaler = MaxAbsScaler()
X_train_LR = scaler.fit_transform(X_train_LR)

In [83]:
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'lbfgs', 'saga'],
    'max_iter': [100,200,300]
}